> # **Dependecies**

In [36]:
#!pip install neattext
#!pip install re

In [ ]:
import pandas as pd               #python package for data analysis
import numpy as np                #python package for handling arrays 
import re                         #python package for dealing with regular expressions
import neattext as nt             #python package for some regex and text cleaninig purposes

> # **Spacy Language Models**

In [ ]:
# should have python 3.6
#!pip install spacy
#!pip install scispacy
#spacy donwnload en_core_web_lg
#spacy download en_ner_bc5cdr_md

In [ ]:
import spacy                            #premium python package for NLP
nlp=spacy.load('en_core_web_lg')        #importing spacy large language model
import scispacy                        #scientific spacy (scispacy) for biomedical uses
sci_nlp=spacy.load(("en_ner_bc5cdr_md") #importing bc5cdr medium model for detection of diseases and drugd

> # **Regular Expressions**

In [ ]:
ind_passport_regex=r'^[A-Z]{2}\d{7}[A-Z]$'
maldv_passport_regex=r'^[A-Z]\d{6}$'
china_passport_regex=r'^[A-Z]\d{8,9}$'
indonsh_passport_regex=r'^[A-Z]\d{9}$'
pak_passport_regex=r'^[A-Z]{2}\d{7}[A-Z]{3}$'

In [ ]:
ind_natid_regex=r'^\d{12}$'
pak_natid_regex=r'^\d{5}\d{7}\d$'
sri_natid_regex=r'^\d{9}[VvXx]$'
bangla_natid_regex=r'^\d{4}\d{2}\d{2}\d{4}\d{5}$'
nepal_natid_regex=r'^\d{10}\d{3}$'

In [ ]:
ssn_regex = r'^\d{3}-\d{2}-\d{4}$'
china_national_id_regex = r'^[A-Z]\d{17}$'
india_national_id_regex = r'^\d{12}$'
japan_national_id_regex = r'^\d{12}$'
pakistan_national_id_regex = r'^\d{13}$'
philippines_national_id_regex = r'^\d{12}$'
ifsc_regex="^[A-Z]{4}0[A-Z0-9]{6}$"

In [ ]:
emails=[]
phone_no=[]
names=[]
drugs=[]
diseases=[]
national_ids=[]
passport_no=[]
gpe=[]
locations=[]
urls=[]
zip_codes=[]
orgs=[]
ip_addresses=[]
mac_addresses=[]
ssn=[]
master_visa_cards=[]
ifsc=[]
fb_profiles=[]
telegram_profiles=[]
twitter_profiles=[]
linkedin_profiles=[]
insta_profiles=[]

> # **Critical PII**

In [ ]:
def extract_critical_pii(data):
    '''
    This function extracts critical PII
    '''
    raw_data=data
    emails.extend(re.findall(re.compile(r'[\w\.-]+@[\w\.-]+', re.UNICODE),raw_data))
    phone_no.extend(re.findall(re.compile(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]',re.UNICODE),raw_data))
    phone_no=[ph_no for ph_no in phone_no if len(ph_no)<=15]
    doc=nlp(data)
    for ent in doc.ents:
        if ent.label_=='PERSON':
            names.append(ent.text)
    doc=sci_nlp(data)
    for ent in doc.ents:
        if ent.label_=='DISEASE':
            dieases.append(ent.text)
        else:
            drugs.append(ent.text)

> # **Sensitive PII**

In [ ]:
def extract_sensitive_pii(data):
    '''
    This function extracts sensitive PII
    '''
    raw_data=data
    for element in data.split('$'):
        ssn.extend(re.findall(r'^\d{3}-\d{2}-\d{4}$',element.strip()))
        ifsc.extend(re.findall(ifsc_regex,element.strip()))
        master_visa_cards.extend(re.findall(re.compile(r'(?:(?:(?:\d{4}[- ]?){3}\d{4}|\d{15,16}))',re.UNICODE),element.strip()))
        master_visa_cards.extend(re.findall(re.compile(r'5[1-5]\d{2}[\s-]?\d{4}[\s-]?\d{4}[\s-]?\d{4}',re.UNICODE),element.strip()))
        passport_no.extend(re.findall(ind_passport_regex,element.strip()))
        passport_no.extend(re.findall(maldv_passport_regex,element.strip()))
        passport_no.extend(re.findall(china_passport_regex,element.strip()))
        passport_no.extend(re.findall(indonsh_passport_regex,element.strip()))
        passport_no.extend(re.findall(pak_passport_regex,element.strip()))
        national_ids.extend(re.findall(ind_natid_regex,element.strip()))
        national_ids.extend(re.findall(pak_natid_regex,element.strip()))
        national_ids.extend(re.findall(sri_natid_regex,element.strip()))
        national_ids.extend(re.findall(bangla_natid_regex,element.strip()))
        national_ids.extend(re.findall(nepal_natid_regex,element.strip()))
        urls.extend(nt.extract_urls(raw_data))
        for url in urls:
            if 'facebook' in url:
                fb_profiles.append(url)
                del urls[url]
            if 't.me' in url:
                telegram_profiles.append(url)
                del urls[url]
            if 'linkedin' in url:
                linkedin_profiles.append(url)
                del urls[url]
            if 'twitter' in url:
                twitter_profiles.append(url)
                del urls[url]
            if 'instagram' in url:
                insta_profiles.append(url)
                del urls[url]

        doc=nlp(raw_data)
        for ent in doc.ents:
            if ent.label_=='GPE':
                gpe.append(ent.text)
            if ent.label_=='LOC':
                locations.append(ent.text)

> # **Quasi PII**

In [ ]:
def extract_quasi_pii(data):
    '''
    This function extracts Quasi PII
    '''
    for element in data.split('$'):
        ip_addresses.extend(re.findall(r"[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}",element.strip()))
        mac_addresses.extend(re.findall(r'^(?:[0-9A-Fa-f]{2}[:-]){5}(?:[0-9A-Fa-f]{2})$',element.strip()))
    doc=nlp(data)
    for ent in doc.ents:
        if ent.label_=='ORG':
            orgs.append(ent.text)

> # **Classification**

In [ ]:
def classification():
    '''
    This function collect all the extracted data and classify into their respective categories
    '''
    critical_pii={'Names':names,'Emails':emails,'Phone No':phone_no,'Diseases':diseases,'Drugs':drugs}
    critical_df=pd.DataFrame(critical_pii)
    sensitive_pii={'SSN':ssn,'Master Visa Cards':master_visa_cards,'IFSC':ifsc,
                   'National ID':national_ids,'Passport No':passport_no,'Telegram Profiles':telegram_profiles,
                  'Twitter Profiles':twitter_profiles,'LinkedIn Profiles':linkedin_profiles,'Instagram Profiles':insta_profiles
                  ,'URLs':urls,'Geographic Locations':gpe,'Locations':locations}
    sensitive_df=pd.DataFrame(sensitive_pii)
    quasi_pii={'Organizations':orgs,'IP Addresses':ip_addresses,'MAC Addresses':mac_addresses}
    quasi=pd.DataFrame(quasi_pii)
    return critical_df,sensitive_df,quasi_pii

> # **Extract PII**

In [35]:
def extract_PII(df):
    '''
    This function receives a dataframe of csv file and format the dataframe and pass to each 
    of critical,sensitive and quasi pii extraction functions.
    '''
    all_data=df.values
    for value in all_data:
        data=' $ '.join(value)
        extract_critical_pii(data)      #passing data to critical pii extraction function
        extract_sensitive_pii(data)     #passing data to sensitive pii extraction function
        extract_quasi_pii(data)         #passing data to quasi pii extraction function

In [ ]:
df=pd.read_csv('file_name.csv')    #reading the csv file

In [ ]:
extract_PII(df)                   #calling extract_PII function

> # **Saving the Extacted Results**

In [ ]:
critical_df.to_csv('critical_pii.csv')
sensitive_df.to_csv('sensitive_pii.csv')
quasi_pii,to_csv('quasi_pii.csv')